### CASE STUDY - CUSTOMER LIFETIME VALUE CALCULATION FOR RETAIL COMPANY


### Tearm and caluclation explanation:

CLTV = (Customer_Value / Churn_Rate) x Profit_margin

CLTV = (Customer_Value * Profit_margin) /  Churn_Rate

- Customer_Value = Average_Order_Value * Purchase_Frequency
- Average_Order_Value = Total_Revenue / Total_Number_of_Orders
- Purchase_Frequency =  Total_Number_of_Orders / Total_Number_of_Customers
- Churn_Rate = 1 - Repeat_Rate
- Also need the Profit_margin

1000 new users. In one year 25% of customers have Retention in one year. What is average lifetime?
- Churn Rate = we will loose 75% of customers in that year.
   - 1 / 0.75 * 12 (Month) = 16 months is average lifetime of a customer
  





### Importing libraries

In [4]:
# importing libraris 

import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 20)
# pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from sklearn.preprocessing import MinMaxScaler

In [5]:
df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")

In [9]:
df = df_.copy()

In [7]:
df.head(100)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom
...,...,...,...,...,...,...,...,...
95,536378,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2010-12-01 09:37:00,2.55000,14688.00000,United Kingdom
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00,0.42000,14688.00000,United Kingdom
97,536378,21975,PACK OF 60 DINOSAUR CAKE CASES,24,2010-12-01 09:37:00,0.55000,14688.00000,United Kingdom
98,536378,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24,2010-12-01 09:37:00,0.55000,14688.00000,United Kingdom


### Data Preparation

In [10]:
#rows with letter C aka cancelled need to be eliminated
canceled_invoice = df[df["Invoice"].str.contains("C", na = False)]

In [11]:
canceled_invoice.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50000,14527.00000,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65000,15311.00000,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65000,17548.00000,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29000,17548.00000,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29000,17548.00000,United Kingdom


In [44]:
canceled_invoice.shape

(9288, 8)

In [45]:
df.shape

(541910, 8)

In [12]:
#new df without cancelled orders
df = df[~df["Invoice"].str.contains("C", na=False)]

In [103]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10000,12680.00000,France,12.60000
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15000,12680.00000,France,16.60000
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15000,12680.00000,France,16.60000
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95000,12680.00000,France,14.85000


In [104]:
df.shape

(397925, 9)

In [13]:
test_number_correct_new_rows = 541910 - 9288
test_number_correct_new_rows

532622

In [15]:
#checking for rows tht have no or negative quantity
df[(df["Quantity"]< 0)].shape

(1336, 8)

In [16]:
#removing rows that have negative quantity
df = df[(df["Quantity"]> 0)]

In [17]:
df.shape

(531286, 8)

In [18]:
#feature generation - total price paid for specific product per order
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [19]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55000,17850.00000,United Kingdom,15.30000
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75000,17850.00000,United Kingdom,22.00000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39000,17850.00000,United Kingdom,20.34000


In [20]:
df.isnull().sum()

Invoice             0
StockCode           0
Description       592
Quantity            0
InvoiceDate         0
Price               0
Customer ID    133361
Country             0
TotalPrice          0
dtype: int64

In [21]:
#removing orders with no customer id or product description
df.dropna(inplace=True)

In [23]:
# one row per customer:
# counting total number of transactions
# summing quantity of each product purchased 
# and total spendings of each customer

cltv_df = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(), 
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

In [25]:
#re-naming the new columns
cltv_df.columns = ['total_transaction', 'total_unit', 'total_price']

In [26]:
# index comes from groupby
cltv_df

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,1,74215,77183.60000
12347.00000,7,2458,4310.00000
12348.00000,4,2341,1797.24000
12349.00000,1,631,1757.55000
12350.00000,1,197,334.40000
...,...,...,...
18280.00000,1,45,180.60000
18281.00000,1,54,80.82000
18282.00000,2,103,178.05000


In [114]:
cltv_df.shape

(4339, 3)

In [27]:
#feature generation: calculating average revenue per user 
cltv_df["avg_order_value"] = cltv_df["total_price"] / cltv_df["total_transaction"]

In [116]:
cltv_df.head()

,total_transaction,total_unit,total_price,avg_order_value
Customer ID,,,,
12346.00000,1,74215,77183.60000,77183.60000
12347.00000,7,2458,4310.00000,615.71429
12348.00000,4,2341,1797.24000,449.31000
12349.00000,1,631,1757.55000,1757.55000
12350.00000,1,197,334.40000,334.40000


In [117]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4339.00000,4.27195,7.70549,1.00000,1.00000,2.00000,5.00000,210.00000
total_unit,4339.00000,1194.21457,5055.60301,1.00000,160.00000,379.00000,993.00000,197491.00000
total_price,4339.00000,2053.79717,8988.24784,0.00000,307.24500,674.45000,1661.64000,280206.02000
avg_order_value,4339.00000,419.04874,1796.32070,0.00000,178.50500,293.69000,430.02250,84236.25000


50% is also called the median. The median and mean are mismatching in this dataset meaning there are likely to be outliers. Median less than mean = right scewed data. Ether many outliers or a few big ones

#### Outlier calculation function

In [29]:
# defining at which value data is considered an outlier. 
# this function will only be called within the function below

def outlier_tresholds(dataframe, col_name, q1 = 0.25, q3 =0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

In [30]:
#calling the function from above inside new function to replace outliers with highest/lowest possible value
# method is called Capping
def replace_with_tresholds(dataframe, variable):
    low_limit, up_limit = outlier_tresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [31]:
#searching/replacing outliers in these columns
outlierTest = ["total_transaction","total_unit","total_price","avg_order_value"]

In [32]:
#calling function defined above
for col in outlierTest:
    replace_with_tresholds(cltv_df, col)

In [33]:
cltv_df.head()

,total_transaction,total_unit,total_price,avg_order_value
Customer ID,,,,
12346.00000,1.00000,2242.50000,3693.23250,807.29875
12347.00000,7.00000,2242.50000,3693.23250,615.71429
12348.00000,4.00000,2242.50000,1797.24000,449.31000
12349.00000,1.00000,631.00000,1757.55000,807.29875
12350.00000,1.00000,197.00000,334.40000,334.40000


In [34]:
cltv_df.shape

(4339, 4)

In [38]:
total_customers = cltv_df.shape[0]

In [36]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_transaction,4339.00000,3.48629,3.04007,1.00000,1.00000,2.00000,5.00000,11.00000
total_unit,4339.00000,685.32968,699.48092,1.00000,160.00000,379.00000,993.00000,2242.50000
total_price,4339.00000,1162.64904,1148.07508,0.00000,307.24500,674.45000,1661.64000,3693.23250
avg_order_value,4339.00000,333.18953,202.65514,0.00000,178.50500,293.69000,430.02250,807.29875


Median and mean are much closer together now. Good! Still not ideal because median still higher than mean.

In [37]:
cltv_df.head()

,total_transaction,total_unit,total_price,avg_order_value
Customer ID,,,,
12346.00000,1.00000,2242.50000,3693.23250,807.29875
12347.00000,7.00000,2242.50000,3693.23250,615.71429
12348.00000,4.00000,2242.50000,1797.24000,449.31000
12349.00000,1.00000,631.00000,1757.55000,807.29875
12350.00000,1.00000,197.00000,334.40000,334.40000


In [40]:
# calculating average number of purchases per client
purchase_frequency = cltv_df['total_transaction'].sum() / total_customers
purchase_frequency

3.4862871629407697

### Calculate repeat rate and churn rate

In [41]:
# calculate rate of returning clients 
repeat_rate = cltv_df[cltv_df['total_transaction'] > 1].shape[0] / total_customers
repeat_rate

0.6556810324959668

In [42]:
churn_rate = 1 - repeat_rate # in one year
churn_rate

0.34431896750403324

65% of customers are considered returning customers while 34% of customers churn after their first purchase

In [50]:
#time customer remains engaged aka Life time
LT = 1/churn_rate 
print(round(LT), "Years users stay active")
print(round(LT*12), "Months users stay active")

3 Years users stay active
35 Months users stay active


### Calculate customer lifetime value

In [51]:
CLTV = LT * purchase_frequency * cltv_df.avg_order_value.mean()*0.15 #0.15 is given profit margin

print("Customer lifetime value for all customers :", CLTV)

Customer lifetime value for all customers : 506.0399691037807


In [58]:
# suppose the customer acquisition cost is 500 pounds
CAC = 500
CLTV_CAC_Ratio = CLTV/CAC

print("The Customer lifetime value to Customer acquisition cost Ratio is: ", round(CLTV_CAC_Ratio, 3))
if CLTV_CAC_Ratio > 1:
    print("Ratio over 1 indicated the business is acquiring customers profitably")
else:
    print("The acquisition costs are too high for the amound of revenue customers are expected to generate therefore it would be more profitable to spend more money in retaining customers")

The Customer lifetime value to Customer acquisition cost Ratio is:  1.012
Ratio over 1 indicated the business is acquiring customers profitably


For every £1 spent on acquiring a customer (CAC), the business expects to earn £1.012 in revenue over the customer's lifetime. This ratio greater than 1 suggests that the business is generating more revenue from each customer than it's spending to acquire them

In [53]:
cltv_df.head() 

,total_transaction,total_unit,total_price,avg_order_value
Customer ID,,,,
12346.00000,1.00000,2242.50000,3693.23250,807.29875
12347.00000,7.00000,2242.50000,3693.23250,615.71429
12348.00000,4.00000,2242.50000,1797.24000,449.31000
12349.00000,1.00000,631.00000,1757.55000,807.29875
12350.00000,1.00000,197.00000,334.40000,334.40000


### Rule Based Segmentation

In order to do specific marketing campaigns for

In [161]:
#splitting the data
cltv_df["segment"] = pd.qcut(cltv_df["avg_order_value"], 4, labels= ["D", "C", "B", "A"])

In [162]:
cltv_df.groupby("segment")[["avg_order_value", "total_transaction"]].agg({"count" , "mean"})

avg_order_value       total_transaction      
                   mean count              mean count
segment                                              
D             123.75338  1085           2.59171  1085
C             230.52516  1085           3.70230  1085
B             352.54848  1084           3.69926  1084
A             625.94896  1085           3.95207  1085

Splitting customers into segments as done above allows businesses to make data-driven decisions tailored to different customer groups. This can lead to more effective marketing, increased customer satisfaction, and ultimately, improved financial performance.

1. **Targeted Marketing**: For example, they can send special offers or promotions to high-value segments (e.g., 'A' segment with high average order value) to maximize revenue and retention, while sending re-engagement offers to lower-value segments.


2. **Personalization**: Understanding customer segments helps in personalizing communication and product recommendations. High-value customers might prefer premium services or products, while budget-conscious customers may appreciate discounts.


3. **Pricing Strategies**: Different segments may respond differently to pricing strategies. Businesses can optimize pricing models for each segment to maximize revenue and profitability.


4. **Product Development**: For instance, high-value segments may indicate opportunities for premium products, while other segments may guide the development of budget-friendly options.


5. **Retention Strategies**: For example, loyalty programs or referral incentives can be tailored to these segments.


6. **Inventory Management**: It can ensure that products preferred by high-value segments are well-stocked and readily available.


7. **Customer Service**:  High-value customers may appreciate proactive support, while others may prefer self-service options.


8. **Predictive Analytics**: For example, businesses can predict which customers are likely to churn and take proactive steps to retain them.


9. **Resource Allocation**: Businesses can allocate marketing budgets, sales efforts, and customer support resources based on the potential return from each segment.

